<a href="https://colab.research.google.com/github/el-grudge/data-engineering-zoomcamp/blob/main/workshop_1/data_talks_club_dlt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [3]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [4]:
# to do: homework :)

## Question 1

- **What is the sum of the outputs of the generator for limit = 5?**

In [5]:
print("Answer 1: {:.3f}".format(sum(square_root_generator(5))))

Answer 1: 8.382


## Question 2

- **What is the 13th number yielded**

In [6]:
print("Answer 2: {:.3f}".format(tuple(enumerate(tuple(square_root_generator(13))))[-1][1]))

Answer 2: 3.606


## Question 3

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [7]:
import dlt
import duckdb

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# we can load any generator to a table at the pipeline destnation as follows:
people_1 = people_1()
info = generators_pipeline.run(people_1,
										table_name="people",
										write_disposition="replace")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n The people table below:")

people_table = conn.sql("SELECT * FROM people").df()
display(people_table)

print("\n\n\n The sum of ages of all people is:")

people_ages = conn.sql("SELECT sum(Age) FROM people").df()
display(people_ages)


Loaded tables: 


┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ _dlt_loads              │
│ _dlt_pipeline_state     │
│ _dlt_version            │
│ people                  │
│ people_with_primary_key │
└─────────────────────────┘




 The people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707711678.3585043,DdfRjIvODUbr4Q,None
1,2,Person_2,27,City_A,1707711678.3585043,WM6Zith9GROl4g,None
2,3,Person_3,28,City_A,1707711678.3585043,4YHSFVcnnat21w,None
3,4,Person_4,29,City_A,1707711678.3585043,5en9HK5Ph/hX8g,None
4,5,Person_5,30,City_A,1707711678.3585043,3/coKr9jpC0icA,None





 The sum of ages of all people is:


,sum(Age)
0,140.0


2. Append the second generator to the same table as the first.

In [8]:
# we can load the next generator to the same or to a different table.
people_2 = people_2()
info = generators_pipeline.run(people_2,
										table_name="people",
										write_disposition="append")

# show outcome
print("\n\n\n The people table below:")

people_table = conn.sql("SELECT * FROM people").df()
display(people_table)




 The people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707711678.3585043,DdfRjIvODUbr4Q,None
1,2,Person_2,27,City_A,1707711678.3585043,WM6Zith9GROl4g,None
2,3,Person_3,28,City_A,1707711678.3585043,4YHSFVcnnat21w,None
3,4,Person_4,29,City_A,1707711678.3585043,5en9HK5Ph/hX8g,None
4,5,Person_5,30,City_A,1707711678.3585043,3/coKr9jpC0icA,None
5,3,Person_3,33,City_B,1707711679.2027369,k/iqL+f5sA2INQ,Job_3
6,4,Person_4,34,City_B,1707711679.2027369,pw8cv9nELaLEqg,Job_4
7,5,Person_5,35,City_B,1707711679.2027369,65drpH/pQJ6evA,Job_5
8,6,Person_6,36,City_B,1707711679.2027369,57mD8JAptBtB0Q,Job_6
9,7,Person_7,37,City_B,1707711679.2027369,NrIJzjZRLdLAPQ,Job_7


3. **After correctly appending the data, calculate the sum of all ages of people.**

In [9]:
print("\n\n\n Answer 3: The sum of ages of all people is:")

people_ages = conn.sql("SELECT sum(Age) FROM people").df()
display(people_ages)




 Answer 3: The sum of ages of all people is:


,sum(Age)
0,353.0


## Question 4

**Calculate the sum of ages of all the people loaded as described above.**


In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


# we can load any generator to a table at the pipeline destnation as follows:
people_1 = people_1()
info = generators_pipeline.run(people_1,
                               table_name="people_with_primary_key",
                               write_disposition="replace",
                               primary_key="ID")

# we can load the next generator to the same or to a different table.
people_2 = people_2()
info = generators_pipeline.run(people_2,
                               table_name="people_with_primary_key",
                               write_disposition="merge",
                               primary_key="ID")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data
print("\n\n\n The people with primary key table below:")

people_table = conn.sql("SELECT * FROM people_with_primary_key").df()
display(people_table)

print("\n\n\n Answer 4: The sum of ages of all people is:")

people_ages = conn.sql("SELECT sum(Age) FROM people_with_primary_key").df()
display(people_ages)


Loaded tables: 


┌─────────────────────────┐
│          name           │
│         varchar         │
├─────────────────────────┤
│ _dlt_loads              │
│ _dlt_pipeline_state     │
│ _dlt_version            │
│ people                  │
│ people_with_primary_key │
└─────────────────────────┘




 The people with primary key table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707711679.866775,iC5q18gz+G08Ew,None
1,2,Person_2,27,City_A,1707711679.866775,B2yL6yEz10AGOQ,None
2,5,Person_5,35,City_B,1707711680.633522,ikp4PBTpamymiw,Job_5
3,7,Person_7,37,City_B,1707711680.633522,kqDHaWi1hAnq+w,Job_7
4,3,Person_3,33,City_B,1707711680.633522,0DHIBV040/DDTw,Job_3
5,6,Person_6,36,City_B,1707711680.633522,SN7uHZ+s7VLeAA,Job_6
6,8,Person_8,38,City_B,1707711680.633522,o7cq+k9vetFH5w,Job_8
7,4,Person_4,34,City_B,1707711680.633522,TV9s8qB4LtU8hw,Job_4





 Answer 4: The sum of ages of all people is:


,sum(Age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX